In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

test_file = '/app/raw/Flights_2018_1.csv'

df = pd.read_csv(test_file, nrows=1000)
# for col in df:
#     print(col)


In [121]:

# df[(df['ArrDel15'] != math.isnan)]
df = df[["DayofMonth","Month","Year", "Distance","DepDelay", "ArrDel15"]].dropna()# get rid of nan
print(df.ArrDel15.unique()) # find unique values

X = df[["DayofMonth","Month","Year", "Distance", "DepDelay"]]
y = df[["ArrDel15"]]
y = y.ArrDel15.ravel() # flatten
print(y.shape)

# split into train and 
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=150) # TODO how to choose random state

[0. 1.]
(915,)


In [125]:
log_reg = LogisticRegression()

#TODO gridCV
# param_grid = [
#   {'solver':[''],'C':[.01, .5, 1, 5, 10, 100, 1000], 'penalty':['l1','l2','elasticnet']},
# ]
param_grid = [
  {'solver':['liblinear'],'C':[.01, .5, .1, 1, 5, 10, 100], 'penalty':['l1','l2']},
  {'solver':['saga'],'C':[.01, .5, .1, 1, 5, 10, 100], 'penalty':['l2','elasticnet']} # l1 gives erros here
]

search = GridSearchCV(log_reg, param_grid, cv=5) # TODO how to choose cv

logreg_model = search.fit(X_train, y_train)

# change this:
print('Grid search found best model solver:', logreg_model.best_estimator_.get_params()['solver'])
print('Grid search found best penalty:', logreg_model.best_estimator_.get_params()['penalty'])
print('Grid search found best hyperparameter C:', logreg_model.best_estimator_.get_params()['C'])

# TODO confirm this is actually printing for best model
# TODO can we get 
test_accuracy = logreg_model.score(X_test, y_test)
train_accuracy = logreg_model.score(X_train, y_train)

print("Test accuracy:" + " " + str(test_accuracy))
print("Train accuracy:" + " " + str(train_accuracy))

/mamba/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/mamba/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/mamba/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/mamba/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/mamba/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/mamba/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not conver

Grid search found best model solver: liblinear
Grid search found best penalty: l1
Grid search found best hyperparameter C: 0.5
Test accuracy: 0.9453551912568307
Train accuracy: 0.9508196721311475


In [ ]:
# NOTE THIS TAKES A FEW MIN TO RUN

# reference: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

# create a pipeline to run standardscaler and then the svm
svm = Pipeline(steps=[("scaler", StandardScaler()), ("svm",SVC())]) # standard scaler makes a huge difference of over 10%
param_grid = [
  {'svm__gamma':['scale', 'auto'],'svm__C':[.01, .5, 1, 5, 10, 100, 1000], "svm__kernel":['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']},
]
search = GridSearchCV(svm, param_grid, cv=5) # TODO how to choose cv
svm_model = search.fit(X_train, y_train)


In [109]:

print('Grid search found best gamma:', svm_model.best_estimator_.get_params()['svm__gamma'])
print('Grid search found best penalty:', svm_model.best_estimator_.get_params()['svm__kernel'])
print('Grid search found best hyperparameter C:', svm_model.best_estimator_.get_params()['svm__C'])

test_accuracy = svm_model.score(X_test, y_test)
train_accuracy = svm_model.score(X_train, y_train)

print("Test accuracy:" + " " + str(test_accuracy))
print("Train accuracy:" + " " + str(train_accuracy))

Grid search found best gamma: scale
Grid search found best penalty: linear
Grid search found best hyperparameter C: 1
Test accuracy: 0.9453551912568307
Train accuracy: 0.9508196721311475
